In [ ]:
import argparse
import gc
import logging
import os
import glob
import pandas as pd

import sys
# sys.path.remove('/opt/ros/kinetic/lib/python2.7/dist-packages')


import time

from collections import defaultdict

import torch
import torch.nn as nn
import torch.optim as optim

import numpy as np
torch.backends.cudnn.benchmark = True

from matplotlib import pyplot as plt
import matplotlib as mpl
import matplotlib.patches as patches
from matplotlib import pyplot as plt
from argoverse.map_representation.map_api import ArgoverseMap
from argoverse.data_loading.argoverse_forecasting_loader import ArgoverseForecastingLoader
from argoverse.visualization.visualize_sequences import viz_sequence
avm = ArgoverseMap()

num = 10

In [ ]:
import os
import sys
sys.path.append("../ddn/")
sys.path.append("./")
import warnings
warnings.filterwarnings('ignore')

import torch
import numpy as np
import scipy.special
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt

from scipy.linalg import block_diag
from torch.utils.data import Dataset, DataLoader
#from bernstein import bernstesin_coeff_order10_new

from argoverse.map_representation.map_api import ArgoverseMap
from argoverse.data_loading.argoverse_forecasting_loader import ArgoverseForecastingLoader
from argoverse.visualization.visualize_sequences import viz_sequence
avm = ArgoverseMap()

def denoise(gt_x, gt_y, w = 7):
    # denoising
    gt_x_t = []
    gt_y_t = []
    for iq in range(len(gt_x)):
        if iq >= w and iq + w <= len(gt_x):
            gt_x_t.append(np.mean(gt_x[iq: iq + w]))
            gt_y_t.append(np.mean(gt_y[iq: iq + w]))
        elif iq < w:
            okx = np.mean(gt_x[w: w + w])
            gt_x_t.append(gt_x[0] + (okx - gt_x[0]) * (iq) / w)
            oky = np.mean(gt_y[w: w + w])
            gt_y_t.append(gt_y[0] + (oky - gt_y[0]) * (iq) / w)
        else:
            okx = np.mean(gt_x[len(gt_x) - w:len(gt_x) - w  + w])
            oky = np.mean(gt_y[len(gt_x) - w: len(gt_x) - w + w])
            gt_x_t.append(okx + (gt_x[-1] - okx) * (w - (len(gt_x) - iq)) / w)
            gt_y_t.append(oky + (gt_y[-1] - oky) * (w - (len(gt_y) - iq)) / w)                   

    gt_x = gt_x_t
    gt_y = gt_y_t
    return gt_x, gt_y

In [ ]:
%matplotlib inline
data_path = "/datasets/argoverse/val/data"
output_dir="../results/"
t_obs=20
output_dir="results/"
dt = 0.3
t_obs=20
pred=False
batch_size = 512
paths = glob.glob(os.path.join(data_path, "*.csv"))
for idx in range(len(paths)):
    if idx <= 169:
        continue
    path = paths[idx]
    df = pd.read_csv(path)
    agent_df = df[df['OBJECT_TYPE'] == 'AGENT']
    others_df = df[df['OBJECT_TYPE'] == 'OTHERS']
    others_dfs = np.array([v for k, v in others_df.groupby('TRACK_ID')], dtype=object)
    av_df = df[df['OBJECT_TYPE'] == 'AV']
    city = df['CITY_NAME'].values[0]

    # agent
    x_traj = agent_df['X'].values
    y_traj = agent_df['Y'].values
    x_a, y_a = x_traj, y_traj
    
    gt_heatmap = np.load("../../intermediate_heatmap_results/{}_gt.npy".format(idx))
    arr = []
    barr = []
    pred_heatmap = np.load("../../intermediate_heatmap_results/{}_predicted.npy".format(idx))
    thres = np.sort(np.unique(pred_heatmap))[-10]
    for i in range(164):
        for j in range(165):
            if pred_heatmap[i][j] > thres:
                arr.append([(j)*100/164 + x_a[20] - 50 ,(165 - i)*100/165 + y_a[20] - 50])
                barr.append([i,j])
#     plt.imshow(pred_heatmap)
#     plt.show()
    plt.figure(figsize=(15,15))    
    arr = np.array(arr)
    plt.scatter(arr[:,0],arr[:,1],color='orange',zorder=8, label='predicted ep')
    total_successors = []
    current = []
    try:
        for indd in range(0, 20):
            lane_id = avm.get_nearest_centerline(np.array([x_traj[indd],y_traj[indd]]), city_name=city)[0].id
            current.append(lane_id)
            successors = avm.get_lane_segment_successor_ids(lane_id, city)
            for successor in successors:
                total_successors.append(successor)
                successors_2d = avm.get_lane_segment_successor_ids(successor, city)
                for successorr in successors_2d:
                    total_successors.append(successorr)
            polygons = [ avm.get_lane_segment_polygon(successor, city) for successor in successors]
        current = np.unique(np.array(current))
        total_successors = np.unique(np.array(total_successors))
        for curr in current:
            current_polygon = avm.get_lane_segment_polygon(curr, city)
            plt.fill(current_polygon[:, 0], current_polygon[:, 1], color='grey')
        for successor in total_successors : 
            polygon = avm.get_lane_segment_polygon(successor, city)
            plt.fill(polygon[:, 0], polygon[:, 1], color='green')
    except:
        print("ok")
#         predecessors = avm.get_lane_segment_predecessor_ids(lane_id, city)        
#         polygons = [ avm.get_lane_segment_polygon(successor, city) for successor in successors]
#         for polygon in polygons : plt.fill(polygon[:, 0], polygon[:, 1], color='grey', label='successor polygons')
    
    x_max, y_max = np.max(x_traj), np.max(y_traj)
    x_min, y_min = np.min(x_traj), np.min(y_traj)

    offsets = [x_traj[0], y_traj[0]] # offsets for other agents
    #         x_traj, y_traj, v_x, v_y, psi, psi_dot, psi_traj, psidot_traj, theta_agent = transform(x_traj, y_traj)

    plt.plot(x_traj[:t_obs], y_traj[:t_obs], color='blue', label='observed')
    plt.scatter(x_traj[t_obs], y_traj[t_obs], color='blue', label='end observed')
    plt.plot(x_traj[t_obs:], y_traj[t_obs:], color='green', label='gt')
    plt.scatter(x_traj[-1], y_traj[-1], color='green', label='gt end point')

    # av
    x_traj = av_df['X'].values
    y_traj = av_df['Y'].values
    x_max, y_max = np.max(x_traj), np.max(y_traj)
    x_min, y_min = np.min(x_traj), np.min(y_traj)

    plt.plot(x_traj, y_traj, color='red', label='AV')
    plt.scatter(x_traj[-1], y_traj[-1], color='red')

    # others
    for other in others_dfs:
        x_traj = other['X'].values
        y_traj = other['Y'].values
        x_max, y_max = np.max(x_traj), np.max(y_traj)
        x_min, y_min = np.min(x_traj), np.min(y_traj)

    #             x_traj, y_traj, v_x, v_y, psi, psi_dot, psi_traj, psidot_traj, theta_others = transform(x_traj, y_traj, theta = theta_agent, offsets = offsets)
        plt.plot(x_traj, y_traj, color='grey')
        plt.scatter(x_traj[-1], y_traj[-1], color='grey')

    # centerlines
    lane_centerlines = []    
    # Get lane centerlines which lie within the range of trajectories
    agent_df = df[df['OBJECT_TYPE'] == 'AGENT']
    gt_x = agent_df['X'].values
    gt_y = agent_df['Y'].values

    x_max, y_max = np.max(gt_x) + 30, np.max(gt_y) + 30
    x_min, y_min = np.min(gt_x) - 30, np.min(gt_y) - 30

    print(x_max, x_min)
    print(y_max, y_min)
    avm = ArgoverseMap()
    for arr in avm.find_local_lane_polygons([x_min, x_max, y_min, y_max], city):
        plt.plot(arr[:, 0], arr[:, 1], color='black')

    seq_lane_props = avm.city_lane_centerlines_dict[city]
    for lane_id, lane_props in seq_lane_props.items():
        lane_cl = lane_props.centerline
        if (np.min(lane_cl[:, 0]) < x_max and np.min(lane_cl[:, 1]) < y_max and np.max(lane_cl[:, 0]) > x_min and np.max(lane_cl[:, 1]) > y_min):
            lane_centerlines.append(lane_cl)

    for lane_cl in lane_centerlines:
        plt.plot(lane_cl[:, 0], lane_cl[:, 1], "--", color="grey", alpha=1, linewidth=1, zorder=0)

    plt.legend()
    plt.xlim([x_a[20] - 50, x_a[20] + 50])
    plt.ylim([y_a[20] - 50, y_a[20] + 50])
    plt.savefig('../../home/{}.png'.format(idx))
#     plt.show()
#     break

In [ ]:
# # predictions = np.load('../prediction.npy', allow_pickle=True)
vis_trajectories(data_path = "/datasets/argoverse/val/data", output_dir="../results/", t_obs=20)

In [ ]:
import pandas as pd
data_ = pd.read_pickle('/datasets/argoverse/forecasting_features_val.pkl')
features = data_["FEATURES"].values
features[0].shape

In [ ]:
for i in range(num):
    city = features[i][0][5]
    seq_lane_props = avm.city_lane_centerlines_dict[city]
    
    gt_x = features[i][:, 3]
    gt_y = features[i][:, 4]
    
    x_max, y_max = np.max(gt_x) + 50, np.max(gt_y) + 50
    x_min, y_min = np.min(gt_x) - 50, np.min(gt_y) - 50
    print(x_max, x_min)
    print(y_max, y_min)
    
    lane_centerlines = []    
    # Get lane centerlines which lie within the range of trajectories
    for lane_id, lane_props in seq_lane_props.items():
        lane_cl = lane_props.centerline

        if (np.min(lane_cl[:, 0]) < x_max and np.min(lane_cl[:, 1]) < y_max and np.max(lane_cl[:, 0]) > x_min and np.max(lane_cl[:, 1]) > y_min):
            lane_centerlines.append(lane_cl)

    for lane_cl in lane_centerlines:
        plt.plot(lane_cl[:, 0], lane_cl[:, 1], "--", color="grey", alpha=1, linewidth=1, zorder=0)
    
    plt.plot(gt_x[:20], gt_y[:20], color='blue', label='observed')
    plt.plot(gt_x[20:], gt_y[20:], color='orange', label='gt')
#     plt.plot(pred_x[20:], pred_y[20:], color='red', label='predicted')
    plt.legend()
    plt.show()
    

In [ ]:
for i in range(num):
    city = features[i][0][5]
    seq_lane_props = avm.city_lane_centerlines_dict[city]
    
    gt_x = features[i][:, 3]
    gt_y = features[i][:, 4]
    
    x_max, y_max = np.max(gt_x) + 50, np.max(gt_y) + 50
    x_min, y_min = np.min(gt_x) - 50, np.min(gt_y) - 50
    print(x_max, x_min)
    print(y_max, y_min)
    
    lane_centerlines = []
    plt.figure(figsize=(10, 10))
    # Get lane centerlines which lie within the range of trajectories
    for arr in avm.find_local_lane_polygons([x_min, x_max, y_min, y_max], city):
        plt.plot(arr[:, 0], arr[:, 1], color='black')
        plt.xlim([x_min, x_max])
        plt.ylim([y_min, y_max])
    for lane_id, lane_props in seq_lane_props.items():
        lane_cl = lane_props.centerline

        if (np.min(lane_cl[:, 0]) < x_max and np.min(lane_cl[:, 1]) < y_max and np.max(lane_cl[:, 0]) > x_min and np.max(lane_cl[:, 1]) > y_min):
            lane_centerlines.append(lane_cl)
    for lane_cl in lane_centerlines:
        plt.plot(lane_cl[:, 0], lane_cl[:, 1], "--", color="grey", alpha=1, linewidth=1, zorder=0)
    plt.plot(gt_x[:20], gt_y[:20], color='blue', label='observed')
    plt.plot(gt_x[20:], gt_y[20:], color='orange', label='gt')
#     plt.plot(pred_x[20:], pred_y[20:], color='red', label='predicted')
    plt.legend()
    plt.show()
    plt.clf()